# Temporal and Spatial DataMining
## Exercise_02_Preprocessing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.style.use('ggplot')

### Task 1: Missing values

**a)** Import the function "get_disturbed_signal" from the util package and plot the returned signal.

In [ ]:
from util import get_disturbed_signal

####################
# Your Code Here   #
####################

**b)** Remove outliers and plot the signal.

**Note:** Think of NaN as well!

In [ ]:
####################
# Your Code Here   #
####################

**c)** Import the function "nan_helper" and replace the missing values by interpolation.
Plot the signal.

**Note:** https://stackoverflow.com/a/6520696

In [ ]:
from util import nan_helper
####################
# Your Code Here   #
####################

### Task 2: PCA (Update: 26.04.2021)

**a)** Extract the "Activity Data" from the zip file. Read the file contents of "u1_s1_rechts.csv" with "pd.read_csv".

In [ ]:
####################
# Your Code Here   #
####################

**b)** Analyse the following function.

In [ ]:
def preprocess_raw(Xraw):
    """ Takes a pandas data frame with raw activity data,
    replaces its labels with numerical values and returns an ndarray 
    without a timestamp.
    
    Parameters:
    ===========
    Xraw - raw data frame
    
    Returns:
    ========
    X,y : ndarray with shape (n,3) [X, Y, Z] and labels
    """
    y = Xraw['Label'].astype('category').cat.codes.values
    X = Xraw[['X','Y','Z']].values
    return X,y

In [ ]:
X,y =  preprocess_raw(Xraw)

**c)** Apply the function given in **b)** to the read data set and plot it.

In [ ]:
####################
# Your Code Here   #
####################

**d)** Read all sensor data and plot each dimension (there are 3 dimensions: x,y,z).

**Note:** Ignore the labels, create one plot for each dimensions containing all sensors.

In [ ]:
Xsraw = [ pd.read_csv("./activity_u1/u1_s{}_rechts.csv".format(i), header=1) for i in range(1,6)]
Xs = [ preprocess_raw(Xr)[0] for Xr in Xsraw ]

In [ ]:
####################
# Your Code Here   #
####################

**e)** Calculate the first 2 principal components using [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) for each sensor. Plot the matching principal components in individial plots.
What do you recognize?

In [ ]:
from sklearn.decomposition import PCA
####################
# Your Code Here   #
####################

### Task 3: Normalization and Standardization

**a)** Import the function "get_random_signal" from the "util" package.
Plot the returned date set using the functions "plot_equal" and "plot_range" from the same package.

In [ ]:
from util import get_random_signal
from util import plot_equal
from util import plot_range

Xnorm = get_random_signal()
####################
# Your Code Here   #
####################

**b)** Implement a function to normalise and a function to standardise the data.
Plot the data in the range (-1,1) for x and y. Use the "apply_plot" function from the "util" package.

In [ ]:
from util import apply_plot
def norm(X):
    #################
    # IMPLEMENT ME! #
    #################
    pass
    ####################
    # Your Code Here   #
    ####################
    
def ztrans(X):
    #################
    # IMPLEMENT ME! #
    #################
    pass
    ####################
    # Your Code Here   #
    ####################

apply_plot(norm, Xnorm)
apply_plot(ztrans, Xnorm)

**c)** Remove all outliers and then normalise and standardize the data set.
Plot the resulting signal.

In [ ]:
####################
# Your Code Here   #
####################

### Task 4: Discretization Quantization
**a)** Read the contents of "u1_s1_rechts.csv" and apply the following function.

In [ ]:
def preprocess_raw(Xraw):
    """ Takes a pandas data frame with raw activity data,
    replaces its labels with numerical values and returns an ndarray 
    without a timestamp.
    
    Parameters:
    ===========
    Xraw - raw data frame
    
    Returns:
    ========
    X,y : ndarray with shape (n,3) [X, Y, Z] and labels
    """
    y = Xraw['Label'].astype('category').cat.codes.values
    X = Xraw[['X','Y','Z']].values
    return X,y

In [ ]:
####################
# Your Code Here   #
####################

**b)** Reduce the size of the data set by 100 using the mean. 

**Note:** Consider the labels! Calculating the mean won't be helpful!

In [ ]:
####################
# Your Code Here   #
####################

**c)** Plot the original and the sampled signal.

In [ ]:
####################
# Your Code Here   #
####################

**d)** Quantise the signal with 32 levels. Implement the required function and plot the resulting signal.

Refer: 
[np.histogram](https://docs.scipy.org/doc/numpy/reference/generated/numpy.histogram.html)
[np.digitize](https://numpy.org/doc/stable/reference/generated/numpy.digitize.html)

In [ ]:
def quantize(X, steps=32):
####################
# Your Code Here   #
####################
    return quant

In [ ]:
# Plot the quantized signal
quant = quantize(disc[:,0])
####################
# Your Code Here   #
####################

**e)** Now, scatter the quantized signal again and use a color coding scheme for the labels.

In [ ]:
####################
# Your Code Here   #
####################

**f)** Save your quantized signal as "xquant.csv". 

In [ ]:
####################
# Your Code Here   #
####################

### Task 5: Fourier Transform

In [ ]:
import matplotlib.pyplot as plt
from util import get_signal
from scipy.stats import norm

**a)** Use the function **get_signal** to import the data with a sampling frequency of **100 Hz**.

It returns two variables x and y

In [ ]:
t, y = get_signal()

**b)** Use the [Fourier Transform](https://numpy.org/devdocs/reference/routines.fft.html) to provide the frequency domain representation of the original signal.

- Consider that the input **y** is a real-valued signal. Use the one-dimensional fourier transform for real input according to this assumption
- Since the input of the fourier-tranform is a time-domain signal, use numpy.abs() to display the amplitude spectrum
- Consider Normalization of the spectrum using the scaling factor $1/N$, $N$: Number of Samples
- Plot the frequency domain representation

In [ ]:
samplingFrequency = 100

####################
# Your Code Here   #
####################

In [ ]:
# Frequency domain representation
plt.figure()
plt.title('Fourier transform depicting the frequency components')
plt.plot(frequencies, np.abs(fourierTransform))

plt.xlabel('Frequency')
plt.ylabel('Amplitude')

plt.show()
### ENDSOLUTION ###

**b)** Apply a low pass filter to the amplitude spectrum to reconstruct the signal and remove high frequency noise terms

In [ ]:
frequency_threshold = 10

####################
# Your Code Here   #
####################

**c)** Apply the inverse Fourier Transform to reconstruct the Signal from the filtered amplitude spectrum

In [ ]:
####################
# Your Code Here   #
####################